In [11]:
import json
import numpy as np
import pandas as pd
from sklearn_crfsuite import CRF
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Bidirectional
from tensorflow.keras.utils import to_categorical
from pyvi import ViTokenizer
import re

# ==============================================================================
# 1. LOAD DỮ LIỆU TỪ FILE BIO
# ==============================================================================
input_file = 'train_bio.json'

print(f"Đang đọc dữ liệu từ {input_file}...")
try:
    with open(input_file, 'r', encoding='utf-8') as f:
        train_sents = json.load(f)
    print(f"-> Đã load thành công {len(train_sents)} câu.")
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file train_bio.json. Hãy đảm bảo bạn đã chạy bước trước đó.")
    exit()

# ==============================================================================
# 2. FEATURE ENGINEERING (TẠO ĐẶC TRƯNG)
# ==============================================================================
def word2features(sent, i):
    word = sent[i][0]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.has_underscore': '_' in word,
    }
    # Ngữ cảnh: Từ trước
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
        })
    else:
        features['BOS'] = True

    # Ngữ cảnh: Từ sau
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
        })
    else:
        features['EOS'] = True
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

# Chuẩn bị dữ liệu cho ML truyền thống
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

# ==============================================================================
# 3. HUẤN LUYỆN 3 MÔ HÌNH TRUYỀN THỐNG
# ==============================================================================
print("\n--- ĐANG TRAIN MÔ HÌNH TRUYỀN THỐNG ---")

# 1. CRF
crf = CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100)
crf.fit(X_train, y_train)
print("✅ CRF: Xong")

# Chuẩn bị dữ liệu phẳng cho LR và RF
dict_vectorizer = DictVectorizer(sparse=False)
X_flat = [item for sublist in X_train for item in sublist]
y_flat = [item for sublist in y_train for item in sublist]
# Fit vectorizer 1 lần duy nhất với dữ liệu train
X_flat_vec = dict_vectorizer.fit_transform(X_flat)

# 2. Logistic Regression
lr = LogisticRegression(max_iter=500, multi_class='ovr') # Dùng OvR cho nhanh
lr.fit(X_flat_vec, y_flat)
print("✅ Logistic Regression: Xong")

# 3. Random Forest
rf = RandomForestClassifier(n_estimators=50, n_jobs=-1) # n_jobs=-1 để chạy đa luồng
rf.fit(X_flat_vec, y_flat)
print("✅ Random Forest: Xong")

# ==============================================================================
# 4. HUẤN LUYỆN DEEP LEARNING (Bi-LSTM)
# ==============================================================================
print("\n--- ĐANG TRAIN DEEP LEARNING (Bi-LSTM) ---")

# Tạo từ điển (Vocabulary & Tags)
words = list(set([t[0] for sent in train_sents for t in sent]))
tags = list(set([t[1] for sent in train_sents for t in sent]))

# Thêm token đặc biệt cho padding và từ lạ (UNK)
if "UNK" not in words: words.append("UNK")
if "PAD" not in words: words.append("PAD") # Padding word

word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
idx2tag = {i: t for t, i in tag2idx.items()}

# Config độ dài
MAX_LEN = 50 
n_words = len(words)
n_tags = len(tags)

# Padding dữ liệu
X_dl = [[word2idx.get(w[0], word2idx["UNK"]) for w in s] for s in train_sents]
X_dl = pad_sequences(X_dl, maxlen=MAX_LEN, padding="post", value=word2idx["PAD"])

y_dl = [[tag2idx[w[1]] for w in s] for s in train_sents]
y_dl = pad_sequences(y_dl, maxlen=MAX_LEN, padding="post", value=tag2idx["O"])
y_dl = [to_categorical(i, num_classes=n_tags) for i in y_dl]

# Xây dựng Model
model_lstm = Sequential([
    Embedding(input_dim=n_words, output_dim=50, input_length=MAX_LEN),
    Bidirectional(LSTM(units=64, return_sequences=True)),
    TimeDistributed(Dense(n_tags, activation="softmax"))
])
model_lstm.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train (Epoch ít để demo nhanh, thực tế nên để 20-50)
model_lstm.fit(X_dl, np.array(y_dl), batch_size=32, epochs=20, verbose=1)
print("✅ Bi-LSTM: Xong")

# ==============================================================================
# 5. HÀM DỰ ĐOÁN & TEST VỚI 3 CÂU MẪU
# ==============================================================================
def predict_and_compare(sentences):
    results = []
    
    for sent in sentences:
        # Tiền xử lý (Giống hệt lúc làm sạch data)
        clean_sent = re.sub(r'[^\w\s\d_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂÂÊÔƠƯưăâêôơư]', ' ', sent)
        clean_sent = re.sub(r'\s+', ' ', clean_sent).strip()
        tokens = ViTokenizer.tokenize(clean_sent).split()
        
        # 1. Predict CRF
        features = [word2features([(t, '') for t in tokens], i) for i in range(len(tokens))]
        pred_crf = crf.predict_single(features)
        
        # 2. Predict LR & RF
        # Quan trọng: Dùng .transform() chứ không fit lại
        vec_features = dict_vectorizer.transform(features) 
        pred_lr = lr.predict(vec_features)
        pred_rf = rf.predict(vec_features)
        
        # 3. Predict Bi-LSTM
        dl_input = [word2idx.get(t, word2idx["UNK"]) for t in tokens]
        dl_input_padded = pad_sequences([dl_input], maxlen=MAX_LEN, padding="post", value=word2idx["PAD"])
        
        pred_prob = model_lstm.predict(dl_input_padded, verbose=0)
        pred_idx = np.argmax(pred_prob, axis=-1)[0]
        
        # Cắt bỏ padding để lấy đúng độ dài câu
        pred_dl = [idx2tag[i] for i in pred_idx][:len(tokens)]
        
        # Lưu kết quả từng từ
        for i, t in enumerate(tokens):
            results.append({
                "Câu": sent[:30] + "...", # Chỉ lấy đoạn đầu làm ID
                "Token": t,
                "CRF": pred_crf[i],
                "LogReg": pred_lr[i],
                "RandForest": pred_rf[i],
                "Bi-LSTM": pred_dl[i]
            })
        
        results.append({"Câu": "---", "Token": "---", "CRF": "---", "LogReg": "---", "RandForest": "---", "Bi-LSTM": "---"})

    return pd.DataFrame(results)


Đang đọc dữ liệu từ train_bio.json...
-> Đã load thành công 248 câu.

--- ĐANG TRAIN MÔ HÌNH TRUYỀN THỐNG ---
✅ CRF: Xong


c:\Users\vthuy\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


✅ Logistic Regression: Xong
✅ Random Forest: Xong

--- ĐANG TRAIN DEEP LEARNING (Bi-LSTM) ---


c:\Users\vthuy\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 59ms/step - accuracy: 0.7589 - loss: 2.3165
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.8776 - loss: 0.9189
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.8776 - loss: 0.6933
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.8776 - loss: 0.6397
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.8776 - loss: 0.5981
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.8776 - loss: 0.5921
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.8776 - loss: 0.5793
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.8776 - loss: 0.5722
Epoch 9/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.8776 - loss: 0.5650
Epoch 10/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8776 - loss: 0.5582
Epoch 11/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.8776 - loss: 0.5506
Epoch 12/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.8776 - loss: 0.5434


In [12]:

# --- DANH SÁCH 3 CÂU TEST ---
test_sentences = [
    'Tác phẩm "Cho tôi xin một vé đi tuổi thơ" của Nguyễn Nhật Ánh ra mắt năm 2008.',
    'Chí Phèo và Thị Nở là hai nhân vật kinh điển của nhà văn Nam Cao.',
    'Bình Ngô đại cáo do Nguyễn Trãi soạn thảo để tuyên cáo chiến thắng quân Minh.'
]

print("\n=== KẾT QUẢ DỰ ĐOÁN ===")
df_result = predict_and_compare(test_sentences)

# Hiển thị đẹp hơn: Chỉ hiện những dòng có ít nhất 1 mô hình tìm ra thực thể (khác O) hoặc dấu gạch ngăn cách
mask = (df_result['CRF'] != 'O') | (df_result['LogReg'] != 'O') | (df_result['RandForest'] != 'O') | (df_result['Bi-LSTM'] != 'O') | (df_result['Token'] == '---')
print(df_result[mask].to_string(index=False))


=== KẾT QUẢ DỰ ĐOÁN ===
                              Câu           Token           CRF        LogReg    RandForest Bi-LSTM
Tác phẩm "Cho tôi xin một vé đ... Nguyễn_Nhật_Ánh         B-PER         B-PER         B-PER       O
Tác phẩm "Cho tôi xin một vé đ...             năm B-TIME / DATE             O   B-TIME/DATE       O
Tác phẩm "Cho tôi xin một vé đ...            2008 I-TIME / DATE I-TIME / DATE I-TIME / DATE       O
                              ---             ---           ---           ---           ---     ---
Chí Phèo và Thị Nở là hai nhân...        Chí_Phèo        B-WORK        B-WORK        B-WORK       O
Chí Phèo và Thị Nở là hai nhân...              và        I-WORK             O             O       O
Chí Phèo và Thị Nở là hai nhân...          Thị_Nở        I-WORK        B-WORK        I-WORK       O
Chí Phèo và Thị Nở là hai nhân...         nhà_văn             O             O         B-PER       O
Chí Phèo và Thị Nở là hai nhân...         Nam_Cao         B-PER         B-P